In [18]:
import pickle
import numpy as np

In [19]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [20]:
# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']

In [21]:
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Load the CIFAR-100 dataset
data_path = 'cifar-100-python/'
train_data_100 = unpickle(data_path + 'train')[b'data']
train_data_100 = train_data_100.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data_100 = unpickle(data_path + 'test')[b'data']
test_data_100 = test_data_100.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels_100 = np.array(unpickle(data_path + 'train')[b'fine_labels'])
test_labels_100 = np.array(unpickle(data_path + 'test')[b'fine_labels'])

In [22]:
# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [23]:
# Print the number of features, classes, and training samples for CIFAR-100
print("\nCIFAR-100")
print("Number of features:", train_data_100.shape[1:])
print("Number of classes:", len(np.unique(train_labels_100)))
print("Number of training samples:", train_data_100.shape[0])


CIFAR-100
Number of features: (32, 32, 3)
Number of classes: 100
Number of training samples: 50000


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
train_data_100, train_labels_100 = shuffle(train_data_100, train_labels_100, random_state=42)

# Print the number of test samples and validation samples for CIFAR-10
print("CIFAR-10 Training Set")
print("Number of training samples:", train_data.shape[0])
print("Number of training labels:", train_labels.shape[0])

# Print the number of test samples and validation samples for CIFAR-100
print("\nCIFAR-100 Training Set")
print("Number of training samples:", train_data_100.shape[0])
print("Number of training labels:", train_labels_100.shape[0])


# Split the test set into test and validation sets for CIFAR-10
test_data, val_data, test_labels, val_labels = train_test_split(test_data, test_labels, test_size=0.5, random_state=42)

# Split the test set into test and validation sets for CIFAR-100
test_data_100, val_data_100, test_labels_100, val_labels_100 = train_test_split(test_data_100, test_labels_100, test_size=0.5, random_state=42)

# Print the number of test samples and validation samples for CIFAR-10
print("\nCIFAR-10 Test Set")
print("Number of test samples:", test_data.shape[0])
print("Number of validation samples:", val_data.shape[0])

# Print the number of test samples and validation samples for CIFAR-100
print("\nCIFAR-100 Test Set")
print("Number of test samples:", test_data_100.shape[0])
print("Number of validation samples:", val_data_100.shape[0])

CIFAR-10 Training Set
Number of training samples: 50000
Number of training labels: 50000

CIFAR-100 Training Set
Number of training samples: 50000
Number of training labels: 50000

CIFAR-10 Test Set
Number of test samples: 5000
Number of validation samples: 5000

CIFAR-100 Test Set
Number of test samples: 5000
Number of validation samples: 5000


In [31]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0
x_val = val_data.astype('float32') / 255.0
x_train_100 = train_data_100.astype('float32') / 255.0
x_test_100 = test_data_100.astype('float32') / 255.0
x_val_100 = val_data_100.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)
y_val = to_categorical(val_labels, num_classes=10)
y_train_100 = to_categorical(train_labels_100, num_classes=100)
y_test_100 = to_categorical(test_labels_100, num_classes=100)
y_val_100 = to_categorical(val_labels_100, num_classes=100)


In [32]:
# Build ResNet-50 model
base_model = ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model_10 = Model(inputs=base_model.input, outputs=predictions)
model_100 = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_10.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Compile the model
model_100.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on CIFAR-10 dataset
model_10.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))
# Train the model on CIFAR-100 dataset
model_100.fit(x_train_100, y_train_100, batch_size=64, epochs=10, validation_data=(x_test_100, y_test_100))

Epoch 1/10
 12/782 [..............................] - ETA: 27:23 - loss: 8.2932 - accuracy: 0.1107

KeyboardInterrupt: 